# Goals
- train predictor on training trusted and untrusted sets 
- use confidence as tampering score

In [1]:
from cupbearer import data, detectors, models, scripts, tasks, utils
from torch.utils.data import DataLoader

/nas/ucb/oliveradk/miniforge3/envs/cupbearer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Train Measurement Predictor

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
os.environ["CUDA_VISIBLE_DEVICES"]

'7'

In [3]:
train_data = data.TamperingDataset("diamonds", train=True)
val_data = data.TamperingDataset("diamonds", train=False)

In [4]:
transformer, tokenizer, emb_dim, max_len = models.transformers_hf.load_transformer(
    "pythia-14m"
)
model = models.TamperingPredictionTransformer(
        model=transformer,
        tokenizer=tokenizer,
        embed_dim=emb_dim,
        max_length=max_len,
        n_sensors=3,
    )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
num_epochs = 5
total_steps = num_epochs * len(train_loader)
scripts.train_classifier( # NOTE: - paper uses 64 warmup steps, but seems hard
    path=(classifier_path := utils.log_path("logs/tampering/predictor")),
    model=model,
    train_loader=train_loader,
    task="multilabel",
    num_labels=4,
    val_loaders=DataLoader(val_data, batch_size=1, shuffle=False),
    lr=2e-5,
    lr_scheduler_conf={
        "warmup_steps": 64, 
        "cosine_annealing": True, 
        "total_steps": total_steps
    },
    max_epochs=num_epochs,
)

/nas/ucb/oliveradk/miniforge3/envs/cupbearer/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /nas/ucb/oliveradk/miniforge3/envs/cupbearer/lib/pyt ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/nas/ucb/oliveradk/miniforge3/envs/cupbearer/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /nas/ucb/oliveradk/miniforge3/envs/cupbearer/lib/pyt ...
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/nas/ucb/oliveradk/miniforge3/envs/cupbearer/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


/nas/ucb/oliveradk/miniforge3/envs/cupbearer/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Epoch 4:  35%|███▌      | 276/782 [02:59<05:29,  1.53it/s, train/loss=7.670]